In [1]:
import numpy as np
import math
from matplotlib import pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import folium
import base64
import cv2
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import random

import random as rand


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
from skimage.color import rgb2gray, rgb2hsv, hsv2rgb
from skimage.io import imread, imshow
from sklearn.cluster import KMeans
from torchsummary import summary as summary

ModuleNotFoundError: No module named 'cv2'

In [2]:
address = "./"
address_weather = "./site_weather_data/value_imputed/"
address_data = "./new_complete_data/complete_data_20192101_20201124/"
address_data_new = "./new_complete_data/신재생(시간별)_complete_data_20191201_20210814/"

batch_size = 32
learning_rate = 1e-4

In [3]:
df_site_info = pd.read_csv(address+'data_each_PV_info_20191201_20210814(bell).csv',index_col = 0)

# PV 전체 데이터 불러와서 Input Output으로 쪼개기

In [4]:
test_df_all = pd.read_csv(address+'pv_all_values.csv',index_col=0)

In [5]:
def build_XY(pv_num):
    X = test_df_all[pv_num].values
    Y = df_site_info.loc[int(pv_num)]['bell']
    
    return np.array([X,Y],dtype='object')

In [ ]:
def save_all_fig():
    for i in range(len(data_set)):
        plt.figure(figsize=(6,3), dpi = 80)
        for j in range(int(len(data_set[i][0])/24)):
            pv_values=data_set[i][0].reshape(-1,24)
            plt.plot(pv_values[j],alpha=0.04)
            plt.axis('off')

        plt.savefig(address+'img_pv/pv_{}_plt.png'.format(df_cols[i]))
        plt.close()

# 저장된 curve image 불러온 후 array로 변환

In [6]:
df_cols = test_df_all.columns.values[2:]
XY = build_XY(df_cols[0])
data_set = np.empty((1,2),dtype='object')
for i,col in enumerate(df_cols):
    XY = build_XY(col)
    data_set = np.insert(data_set,-1,XY,axis=0)
data_set=np.delete(data_set,-1,axis=0)
data_set[:,1] = data_set[:,1]*1

save_all_fig()

for i in range(len(data_set)):
    img = Image.open('img_pv/pv_{}_plt.png'.format(df_cols[i])).convert('L')
    img_arr = np.array(img, dtype='uint8')
    img_shape = img_arr.shape
    img_arr = img_arr.reshape((1,img_shape[0],img_shape[1]))
    data_set[i][0]=img_arr

In [9]:
dataset_len = data_set.shape[0]
dataset_train = data_set[:int(dataset_len*0.7)].tolist()
dataset_valid =data_set[int(dataset_len*0.7):int(dataset_len*0.9)].tolist()
dataset_test =data_set[int(dataset_len*0.9):].tolist()

dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, sampler=None, batch_sampler=None, num_workers=0, drop_last=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=batch_size, shuffle=None, sampler=None, batch_sampler=None, num_workers=0, drop_last=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=None, sampler=None, batch_sampler=None, num_workers=0, drop_last=True)

# Regular Curve Detection Model 

In [12]:
class CAE_Network(nn.Module):
    def __init__(self,feature_size,out_size,kernel_size):
      super(CAE_Network, self).__init__()

      self.conv1_layer = nn.Sequential(
          nn.Conv2d(1,out_size,kernel_size,padding=int(kernel_size/2)),
          nn.BatchNorm2d(out_size),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d(kernel_size = kernel_size,stride=2,padding=int(kernel_size/2))   # pooling layer to reduce x-y dims by two; kernel and stride of 2
      )
      self.conv2_layer = nn.Sequential(
          nn.Conv2d(out_size,out_size**2,kernel_size,padding=int(kernel_size/2)),
          nn.BatchNorm2d(out_size**2),
          nn.LeakyReLU(inplace=True),
          nn.MaxPool2d(kernel_size = kernel_size,stride=2,padding=int(kernel_size/2))
      )
#       self.conv3_layer = nn.Sequential(
#           nn.Conv2d(out_size**2,out_size**3,3,padding=1),
#           nn.BatchNorm2d(out_size**3),
#           nn.LeakyReLU(inplace=True),
#           nn.MaxPool2d(kernel_size = 3,stride=2,padding=1)
#       )
      self.feature_extraction = nn.Sequential(
          nn.Linear(4*450*4*(out_size**2),feature_size)
      )
      self.sigmoid_prob = nn.Sequential(
          nn.Linear(feature_size,2),
          nn.ReLU(inplace=True),
          nn.Sigmoid()
      )
        
    def flatten_feature(self,x):
      out = torch.flatten(x,1)
      return out
    
    def forward(self,x):

      x = self.conv1_layer(x)
      x = self.conv2_layer(x)
#       x = self.conv3_layer(x)
      o = self.flatten_feature(x) 
      o = self.feature_extraction(o)
      o = F.leaky_relu(o,inplace=True)
      o = self.sigmoid_prob(o)
      return o

# Parameter test

In [15]:
param_set = [[16,3,5],[16,3,3],[16,2,5],[16,2,3],[16,4,3]
            ,[8,3,5],[8,3,3],[8,2,5],[8,2,3],[8,4,5],[8,4,3]
            ,[64,3,5],[64,3,3],[64,2,5],[64,2,3],[64,4,5],[64,4,3]
            ,[32,3,5],[32,3,3],[32,2,5],[32,2,3],[32,4,5],[32,4,3]]

In [ ]:
for param in param_set:
    
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    encoder = CAE_Network(feature_size = param[0],out_size = param[1], kernel_size =param[2])
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate, weight_decay = 1e-3)
    criterion = nn.CrossEntropyLoss()
    encoder.to(device)
    print("----------------------------------------------------------------")
    print("Feature Size\t:\t",param[0])
    print("Out Size\t:\t",param[1])
    print("Kernel Size\t:\t",param[2])
    print(summary(encoder, (1,240,480))) # (model, input_size)


    loss_train = []
    mape_temp = math.inf
    n_epochs = 5

    for epoch in range(n_epochs):
        train_pred = []
        train_true = []
        encoder.train()
        for batch_idx, (data, label) in enumerate(dataloader_train):
            data, label = Variable(data)/255, Variable(label).long() 
            data = data.to(device)
            label = label.to(device)
            encoder_output = encoder(data)
            pred = encoder_output
            _, pred_acc = torch.max(encoder_output,1)

            train_pred.append(pred_acc.detach().numpy().reshape(-1))
            train_true.append(label.cpu().detach().numpy().reshape(-1))    

            ##
            loss = 0
            loss += criterion(pred, label)

            encoder_optimizer.zero_grad()
            loss.backward()
            encoder_optimizer.step()

            if batch_idx%50 == 0:
                print("epoch %d : batch_idx %d, loss %.6f" 
                      %(epoch, batch_idx, loss.item()))
                loss_train.append(loss.item())

        train_pred = np.array(train_pred, dtype = 'float32')
        train_pred = train_pred.reshape(-1,1)

        train_true = np.array(train_true, dtype = 'float32')
        train_true = train_true.reshape(-1,1)

        train_acc_sum = np.sum(train_pred == train_true)
        train_acc = train_acc_sum / len(train_true) *100

        print(len(train_true))
        print("epoch %d : train Acc : %.2f" %(epoch, train_acc))
        print("======================================================================================")

        #validation
        encoder.eval()
        valid_pred = []
        valid_true = []
        for batch_idx_v, (data_v, label_v) in enumerate(dataloader_valid):
            data_v, label_v = Variable(data_v)/255, Variable(label_v).long()
            data_v = data_v.to(device)
            label_v = label_v.to(device)

            encoder_output_v = encoder(data_v)
            _, pred_v = torch.max(encoder_output_v,1)

            valid_pred.append(pred_v.detach().numpy().reshape(-1))
            valid_true.append(label_v.cpu().detach().numpy().reshape(-1))

        valid_pred = np.array(valid_pred, dtype = 'float32')
        valid_pred = valid_pred.reshape(-1,1)

        valid_true = np.array(valid_true, dtype = 'float32')
        valid_true = valid_true.reshape(-1,1)

        acc_sum = np.sum(valid_pred == valid_true)
        acc = acc_sum / len(valid_true) *100

        print(len(valid_true))
        print("epoch %d : Validation Acc : %.2f" %(epoch, acc))
        print("======================================================================================")
        print("======================================================================================")
    print("done!!")

----------------------------------------------------------------
Feature Size	:	 16
Out Size	:	 3
Kernel Size	:	 5
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 3, 240, 480]              78
       BatchNorm2d-2          [-1, 3, 240, 480]               6
         LeakyReLU-3          [-1, 3, 240, 480]               0
         MaxPool2d-4          [-1, 3, 120, 240]               0
            Conv2d-5          [-1, 9, 120, 240]             684
       BatchNorm2d-6          [-1, 9, 120, 240]              18
         LeakyReLU-7          [-1, 9, 120, 240]               0
         MaxPool2d-8           [-1, 9, 60, 120]               0
            Linear-9                   [-1, 16]       1,036,816
           Linear-10                    [-1, 2]              34
             ReLU-11                    [-1, 2]               0
          Sigmoid-12                    [-1, 2]     

epoch 0 : batch_idx 0, loss 0.691808, teacher forcing ratio : 0.50
use teacher forcing : False
epoch 0 : batch_idx 50, loss 0.550864, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 0 : train Acc : 92.50
480
epoch 0 : Validation Acc : 97.71
epoch 1 : batch_idx 0, loss 0.584183, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 1 : batch_idx 50, loss 0.557095, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 1 : train Acc : 98.81
480
epoch 1 : Validation Acc : 99.17
epoch 2 : batch_idx 0, loss 0.556938, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 2 : batch_idx 50, loss 0.529230, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 2 : train Acc : 99.03
480
epoch 2 : Validation Acc : 99.17
epoch 3 : batch_idx 0, loss 0.563270, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 3 : batch_idx 50, loss 0.529025, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 3 : train Acc : 99.2

epoch 0 : batch_idx 50, loss 0.475461, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 0 : train Acc : 86.70
480
epoch 0 : Validation Acc : 100.00
epoch 1 : batch_idx 0, loss 0.487194, teacher forcing ratio : 0.50
use teacher forcing : False
epoch 1 : batch_idx 50, loss 0.474204, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 1 : train Acc : 99.26
480
epoch 1 : Validation Acc : 99.79
epoch 2 : batch_idx 0, loss 0.474293, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 2 : batch_idx 50, loss 0.486878, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 2 : train Acc : 99.26
480
epoch 2 : Validation Acc : 99.79
epoch 3 : batch_idx 0, loss 0.474185, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 3 : batch_idx 50, loss 0.474157, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 3 : train Acc : 99.38
480
epoch 3 : Validation Acc : 99.79
epoch 4 : batch_idx 0, loss 0.474134, teacher forcing 

1760
epoch 0 : train Acc : 94.83
480
epoch 0 : Validation Acc : 99.79
epoch 1 : batch_idx 0, loss 0.474282, teacher forcing ratio : 0.50
use teacher forcing : False
epoch 1 : batch_idx 50, loss 0.474572, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 1 : train Acc : 99.38
480
epoch 1 : Validation Acc : 99.58
epoch 2 : batch_idx 0, loss 0.489883, teacher forcing ratio : 0.50
use teacher forcing : False
epoch 2 : batch_idx 50, loss 0.474308, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 2 : train Acc : 99.38
480
epoch 2 : Validation Acc : 99.58
epoch 3 : batch_idx 0, loss 0.474658, teacher forcing ratio : 0.50
use teacher forcing : False
epoch 3 : batch_idx 50, loss 0.474191, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 3 : train Acc : 99.38
480
epoch 3 : Validation Acc : 99.79
epoch 4 : batch_idx 0, loss 0.474176, teacher forcing ratio : 0.50
use teacher forcing : False
epoch 4 : batch_idx 50, loss 0.477330, teacher forcin

480
epoch 0 : Validation Acc : 99.17
epoch 1 : batch_idx 0, loss 0.490383, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 1 : batch_idx 50, loss 0.474384, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 1 : train Acc : 99.38
480
epoch 1 : Validation Acc : 99.58
epoch 2 : batch_idx 0, loss 0.474370, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 2 : batch_idx 50, loss 0.474377, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 2 : train Acc : 99.38
480
epoch 2 : Validation Acc : 99.58
epoch 3 : batch_idx 0, loss 0.474265, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 3 : batch_idx 50, loss 0.489904, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 3 : train Acc : 99.38
480
epoch 3 : Validation Acc : 99.58
epoch 4 : batch_idx 0, loss 0.474325, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 4 : batch_idx 50, loss 0.474214, teacher forcing ratio : 0.50
use teacher forcing : T

...

epoch 1 : batch_idx 50, loss 0.599237, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 1 : train Acc : 40.91
480
epoch 1 : Validation Acc : 48.33
epoch 2 : batch_idx 0, loss 0.604150, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 2 : batch_idx 50, loss 0.562089, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 2 : train Acc : 40.91
480
epoch 2 : Validation Acc : 48.33
epoch 3 : batch_idx 0, loss 0.631534, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 3 : batch_idx 50, loss 0.583612, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 3 : train Acc : 40.91
480
epoch 3 : Validation Acc : 48.33
epoch 4 : batch_idx 0, loss 0.583614, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 4 : batch_idx 50, loss 0.604778, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 4 : train Acc : 40.85
480
epoch 4 : Validation Acc : 48.33
done!!
-----------------------------------------------

1760
epoch 1 : train Acc : 99.38
480
epoch 1 : Validation Acc : 99.58
epoch 2 : batch_idx 0, loss 0.474113, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 2 : batch_idx 50, loss 0.474126, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 2 : train Acc : 99.38
480
epoch 2 : Validation Acc : 99.58
epoch 3 : batch_idx 0, loss 0.474132, teacher forcing ratio : 0.50
use teacher forcing : False
epoch 3 : batch_idx 50, loss 0.474138, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 3 : train Acc : 99.43
480
epoch 3 : Validation Acc : 99.58
epoch 4 : batch_idx 0, loss 0.474132, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 4 : batch_idx 50, loss 0.474115, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 4 : train Acc : 99.43
480
epoch 4 : Validation Acc : 99.58
done!!
----------------------------------------------------------------
Feature Size	:	 64
Out Size	:	 4
Kernel Size	:	 5
-----------------------------

480
epoch 1 : Validation Acc : 48.33
epoch 2 : batch_idx 0, loss 0.693147, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 2 : batch_idx 50, loss 0.693147, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 2 : train Acc : 40.91
480
epoch 2 : Validation Acc : 48.33
epoch 3 : batch_idx 0, loss 0.693147, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 3 : batch_idx 50, loss 0.693147, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 3 : train Acc : 40.91
480
epoch 3 : Validation Acc : 48.33
epoch 4 : batch_idx 0, loss 0.693147, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 4 : batch_idx 50, loss 0.693147, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 4 : train Acc : 40.85
480
epoch 4 : Validation Acc : 48.33
done!!
----------------------------------------------------------------
Feature Size	:	 32
Out Size	:	 3
Kernel Size	:	 5
----------------------------------------------------------------

epoch 2 : batch_idx 0, loss 0.474534, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 2 : batch_idx 50, loss 0.474204, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 2 : train Acc : 99.38
480
epoch 2 : Validation Acc : 99.58
epoch 3 : batch_idx 0, loss 0.489922, teacher forcing ratio : 0.50
use teacher forcing : False
epoch 3 : batch_idx 50, loss 0.474145, teacher forcing ratio : 0.50
use teacher forcing : False
1760
epoch 3 : train Acc : 99.38
480
epoch 3 : Validation Acc : 99.79
epoch 4 : batch_idx 0, loss 0.474130, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 4 : batch_idx 50, loss 0.474164, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 4 : train Acc : 99.38
480
epoch 4 : Validation Acc : 99.79
done!!
----------------------------------------------------------------
Feature Size	:	 32
Out Size	:	 2
Kernel Size	:	 5
----------------------------------------------------------------
        Layer (type)             

epoch 2 : batch_idx 50, loss 0.489881, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 2 : train Acc : 99.32
480
epoch 2 : Validation Acc : 99.79
epoch 3 : batch_idx 0, loss 0.487713, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 3 : batch_idx 50, loss 0.489815, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 3 : train Acc : 99.38
480
epoch 3 : Validation Acc : 99.79
epoch 4 : batch_idx 0, loss 0.474170, teacher forcing ratio : 0.50
use teacher forcing : True
epoch 4 : batch_idx 50, loss 0.489817, teacher forcing ratio : 0.50
use teacher forcing : True
1760
epoch 4 : train Acc : 99.38
480
epoch 4 : Validation Acc : 99.79
done!!
----------------------------------------------------------------
Feature Size	:	 32
Out Size	:	 4
Kernel Size	:	 5
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 4, 240, 480]             104
  

In [13]:
def CAE_model_load():
    model = CAE_Network(feature_size = 128,out_size = 3)
    PATH = "CAE_bell_classifier.pth"
    model.load_state_dict(torch.load(PATH))
    model.eval()
    return model

In [14]:
def return_bell(model,input_data):
    bell_tf = torch.max(model(input_data),1)[1]
    return bell_tf

# Bell Shape 판별해서 예측돌리는 코드

In [ ]:
# data_PV_Info = pd.read_csv(address+'data_each_PV_info_20191201_20210814(bell).csv')
# PV_site_isin = (np.equal(data_PV_Info["광역지역"]==site_korean , data_PV_Info["bell"]))
# data_PV_Info_site = data_PV_Info[PV_site_isin][data_PV_Info[PV_site_isin]['광역지역']==site_korean]



# info_num_array = data_PV_Info_site['번호'].values
# info_add_array = data_PV_Info_site['발전기명'].values
# info_cap_array = data_PV_Info_site['설비용량(MW)'].values
# info_lat_array = data_PV_Info_site['세부지역'].values
# info_column_name = ['Number','Address','Latitude and Longitude','Capacity']

# info_set = np.vstack((info_num_array,info_add_array,info_lat_array,info_cap_array)).T

# df_site_info =pd.DataFrame(info_set, columns = info_column_name)
# df_site_info.to_csv('PV_'+site+'_info_bell.csv',encoding='utf-8-sig')

# column_name=['date','hour']
# print(site_korean)
# print(len(data_PV_Info_site))


# for i in range(len(data_PV_Info_site)):
#     site_name = data_PV_Info_site['발전기명'].values[i]
#     site_num = data_PV_Info_site['번호'].values[i]
#     site_cap = data_PV_Info_site['설비용량(MW)'].values[i]

#     column_name.append(str(int(site_num)))
# #     print("Site Name :",site_name," / Site Num :",site_num," / Site Capacity :",site_cap)

#     data_PV_Value = pd.read_csv(address_data_new+'PV_'+site_name+'_20191201_20210814.csv')

#     hours = data_PV_Value['시간대(1~24h)'].values
#     date = data_PV_Value['거래일'].values
#     gen_amount = data_PV_Value['발전량'].values

#     if i != 0:
#         value_set = np.vstack((value_set,gen_amount))
#     else : value_set = np.vstack((date.T, hours, gen_amount))


# value_set = value_set.T

# df_site_value = pd.DataFrame(value_set, columns = column_name)
# df_site_value.to_csv('PV_'+site+'_value_bell.csv',encoding='utf-8-sig')

# data_PV_Info = pd.read_csv(address+'PV_'+site+'_info_bell.csv',index_col=0)
# data_PV_Value = pd.read_csv(address+'PV_'+site+'_value_bell.csv',index_col=0)

In [11]:
# site_set = ["seoul","choongnam","choongbook","busan","daegoo","daejeon","woolsan","incheon","kssouth","ksnorth","kyoungki","gangwon"]
site_set = ["kwangjoo","sejong","choongnam","seoul","busan","daegoo","choongbook","daejeon","woolsan","incheon","kssouth","ksnorth","kyoungki","gangwon","jn","jb","jeju"]

test_month_array =['19_12','20_1','20_2','20_3','20_4','20_5','20_6','20_7','20_8',
                   '20_9','20_10','20_11','20_12','21_1','21_2','21_3','21_4','21_5',
                   '21_6','21_7','21_8']
epsilon = 10e-5

for p in range(len(test_month_array)):
    site_forecast_agg_result_array = []
    test_month = test_month_array[p]
    print("-"*80)
    print("-"*80)
    print("-"*80)
    print("-"*80)
    print("Test Month: ",test_month)
    print("-"*80)
    print("-"*80)
    print("-"*80)
    print("-"*80)
    for k in range(len(site_set)):

        site = site_set[k]

        if site == "gangwon":
            site_korean = "강원"
            weather_korean = "강릉"
        elif site =="seoul":
            site_korean = "서울"
            weather_korean = "서울"
        elif site =="choongnam":
            site_korean = "충남"
            weather_korean = "천안"
        elif site =="choongbook":
            site_korean = "충북"
            weather_korean = "청주"
        elif site =="sejong":
            site_korean = "세종"
            weather_korean = "세종"
        elif site =="busan":
            site_korean = "부산"
            weather_korean = "부산"
        elif site =="kyoungki":
            site_korean = "경기"
            weather_korean = "수원"
        elif site =="kwangjoo":
            site_korean = "광주"
            weather_korean = "광주"
        elif site =="daegoo":
            site_korean = "대구"
            weather_korean = "대구"
        elif site =="daejeon":
            site_korean = "대전"
            weather_korean = "대전"
        elif site =="woolsan":
            site_korean = "울산"
            weather_korean = "울산"
        elif site =="incheon":
            site_korean = "인천"
            weather_korean = "인천"
        elif site =="kssouth":
            site_korean = "경남"
            weather_korean = "김해시"
        elif site =="ksnorth":
            site_korean = "경북"
            weather_korean = "경주시"
        elif site =="jn":
            site_korean = "전남"
            weather_korean = "장흥"
        elif site =="jb":
            site_korean = "전북"
            weather_korean = "장수"
        elif site =="jeju":
            site_korean = "제주"
            weather_korean = "제주"

        data_PV_Info = pd.read_csv(address+'data_each_PV_info_20191201_20210814(bell).csv')
        PV_site_isin = (np.equal(data_PV_Info["광역지역"]==site_korean , data_PV_Info["bell"]))
        data_PV_Info_site = data_PV_Info[PV_site_isin][data_PV_Info[PV_site_isin]['광역지역']==site_korean]

        data_PV_Info = pd.read_csv(address+'PV_'+site+'_info_bell.csv',index_col=0)
        data_PV_Value = pd.read_csv(address+'PV_'+site+'_value_bell.csv',index_col=0)

        ###############################################################################
        
        month = [31, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31,28,31,30,31,30,31,14] 
        train_len1 = sum(month[:month_test])
        train_len2 = sum(month[month_test+1:])
        train_len = train_len1 + train_len2
        ###############################################################################
        train_input  = []
        train_output = []
        test_input   = []
        test_output  = []

        # Input Data (입력값을 배열로 만드는 과정)
        data_Weather = pd.read_csv(address+'OBS_ASOS_TIM_merged_'+weather_korean+'_imputed.csv')
        data_Temp = np.nan_to_num(np.reshape(np.array(data_Weather['Temp']), (623,24)))/40
        data_Humid = np.nan_to_num(np.reshape(np.array(data_Weather['Humid']), (623,24)))/100
        data_Wind = np.nan_to_num(np.reshape(np.array(data_Weather['Wind']), (623,24)))/20
        data_Cloud = np.nan_to_num(np.reshape(np.array(data_Weather['Cloud']), (623,24)))/10

        Input = np.concatenate((data_Temp, data_Humid, data_Wind, data_Cloud), axis=1)
        ###############################################################################
        for l in range(len(data_PV_Info)):
            Num = data_PV_Info['Number'][l]
            Cap = data_PV_Info['Capacity'][l]

            data_PV24 = np.zeros((623, 24)) # 날짜의 길이에 따라 623이 360로 바뀔 수 있음.
            j = 0
            for i in range(len(data_PV_Value)):
                if i != 0 and data_PV_Value['date'][i] != data_PV_Value['date'][i-1]:
                    j += 1
                h = int(data_PV_Value['hour'][i] - 1)
                data_PV24[j][h] = data_PV_Value[str(Num)][i]

            if l==0: Output = data_PV24/Cap
            else: Output = np.append(Output, data_PV24/Cap, axis=1)
        
        if p == len(test_month_array)-1:
            train_input = Input[:train_len1]
            train_output = Output[:train_len1]
        else:
            train_input = np.append(Input[:train_len1], Input[-train_len2:], axis=0)
            train_output = np.append(Output[:train_len1], Output[-train_len2:], axis=0)
        test_input = Input[train_len1:train_len1 + month[month_test]]
        test_output = Output[train_len1:train_len1 + month[month_test]]  
        
        # Model Spec (뉴런 개수와 레이어 수는 바꿔보면서 Test)
        in_size        = len(train_input[0])
        hidden_size    = 256
        out_size       = len(train_output[0])
        batch_size     = 32
        learning_rate  = 1e-5

        class MLP(nn.Module):
            def __init__(self):
                super(MLP, self).__init__()
                self.fc_in  = nn.Linear(in_size, hidden_size)
                self.fc_h1  = nn.Linear(hidden_size, hidden_size)
                self.fc_h2  = nn.Linear(hidden_size, hidden_size)
                self.fc_out = nn.Linear(hidden_size, out_size)

                torch.nn.init.xavier_uniform_(self.fc_in.weight)
                torch.nn.init.xavier_uniform_(self.fc_h1.weight)
                torch.nn.init.xavier_uniform_(self.fc_h2.weight)
                torch.nn.init.xavier_uniform_(self.fc_out.weight)

            def forward(self, x):
                x = F.relu(self.fc_in(x))
                x = F.relu(self.fc_h1(x))
                x = F.relu(self.fc_h2(x))
                out = F.relu(self.fc_out(x))
                return out

        def train_net(model, batch, optimizer):
            x, y = batch[0], batch[1]
            loss = F.mse_loss(model.forward(x), y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Model Training (Total epoch 수도 적절하게 변경)
        # NMAE_test를 보면 오차가 2~3%대까지 내려갈 수 있는데 이는 밤 시간 데이터가 포함된 오차
        # 우리 관심사인 낮 시간 (이용률 = 발전량/설비용량 이 10% 이상인 것)에서의 오차는 이보다 클 수 있으니 주의
        total_epoch    = 1000
        print_interval = 100

        model = MLP()

        total_batch = int(train_len/batch_size) + 1
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        train_predict = []
        test_predict = []
        NMAE_train = []
        NMAE_test  = []
        print("-"*40)
        print("-"*40)
        print("예측 지역 :",site_korean)
        for epoch in range(total_epoch):
            for i in range(total_batch):
                batch_x = torch.tensor(train_input[batch_size*i:batch_size*(i+1),:], dtype=torch.float)
                batch_y = torch.tensor(train_output[batch_size*i:batch_size*(i+1),:], dtype=torch.float)
                batch = [batch_x, batch_y]
                train_net(model, batch, optimizer)

            if epoch == 0 or (epoch+1) % print_interval == 0:
                train_predict += [model.forward(torch.tensor(train_input, dtype=torch.float)).detach().numpy()]
                test_predict  += [model.forward(torch.tensor(test_input, dtype=torch.float)).detach().numpy()]
                NMAE_train += [round(100*np.mean(np.abs(train_output - train_predict)),2)]
                NMAE_test  += [round(100*np.mean(np.abs(test_output - test_predict)),2)]

                print("epoch: {}".format(epoch+1).ljust(25), end="")
                print("NMAE_train: {}%".format(NMAE_train[-1]).ljust(25), end="")
                print("NMAE_test: {}%".format(NMAE_test[-1]).ljust(25))

        i = np.argmin(NMAE_test) # test를 validation처럼 활용한 것
        # 아래는 데이터를 각 지역별로 쪼개서 오차 확인하고, 쪼갠 데이터를 Aggregation 한 후 오차 확인하는 것
        # 낮 시간에서의 오차를 5%대로 만드는 것이 목표
        test_output_each = []
        test_predict_each = []
        test_output_agg = np.zeros((np.size(test_output,0), 24))
        test_predict_agg = np.zeros((np.size(test_predict[i],0), 24))
        for j in range(len(data_PV_Info)):
            test_output_each += [test_output[:,24*j:24*(j+1)]]
            test_predict_each += [test_predict[i][:,24*j:24*(j+1)]]
            test_output_agg += data_PV_Info['Capacity'][j]*test_output_each[j]
            test_predict_agg += data_PV_Info['Capacity'][j]*test_predict_each[j]
        test_output_agg = test_output_agg/sum(data_PV_Info['Capacity'])
        test_predict_agg = test_predict_agg/sum(data_PV_Info['Capacity'])

        site_forecast_result_array = []
        for j in range(len(data_PV_Info)):
            test_object = test_output_each[j] > 0.1 # 이용률이 10% 이상인 것만 선정
            NMAE_test_object = round(100*np.sum(test_object*np.abs(test_output_each[j] - test_predict_each[j]))/(np.sum(test_object)+epsilon),2)
            print(data_PV_Info['Address'][j] + ": {}%".format(NMAE_test_object).ljust(25))
            site_forecast_result_array.append([data_PV_Info['Address'][j], "{}%".format(NMAE_test_object)])
        
        if not os.path.isdir(address+'광역예측결과_논문/{}월'.format(test_month)):
            os.mkdir(address+'광역예측결과_논문/{}월'.format(test_month))    
        
        df_site_forecast_result = pd.DataFrame(site_forecast_result_array, columns=['each Address','NMAE'])
        df_site_forecast_result.to_csv(address+'광역예측결과_논문/{}월/{}{}_NMAE_each_result.csv'.format(test_month,test_month,site_korean),encoding='utf-8-sig')

        test_object = test_output_agg > 0.1 # Aggregation 후에도 이용률이 10% 이상인 것만 선정
        NMAE_test_object = round(100*np.sum(test_object*np.abs(test_output_agg - test_predict_agg))/(np.sum(test_object)+epsilon),2)
        print("Aggregation: {}%".format(NMAE_test_object).ljust(25))

        site_forecast_agg_result_array.append([site_korean,"{}%".format(NMAE_test_object)])

        print("-"*40)
        print("-"*40)
    
    df_site_agg_result = pd.DataFrame(site_forecast_agg_result_array, columns=['Wide Address','NMAE'])
    df_site_agg_result.to_csv(address+'광역예측결과_논문/{}월/{}_Wide_NMAE_result.csv'.format(test_month,test_month),encoding='utf-8-sig')


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Test Month:  19_12
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
----------------------------------------
----------------------------------------
예측 지역 : 광주
epoch: 1                 NMAE_train: 15.04%       NMAE_test: 10.77%        
epoch: 100               NMAE_train: 10.47%       NMAE_test: 7.64%         
epoch: 200               NMAE_train: 8.64%        NMAE_test: 6.35%         
epoch: 300  

----------------------------------------
----------------------------------------
예측 지역 : 서울
epoch: 1                 NMAE_train: 14.88%       NMAE_test: 10.8%         
epoch: 100               NMAE_train: 10.58%       NMAE_test: 8.01%         
epoch: 200               NMAE_train: 8.85%        NMAE_test: 6.74%         
epoch: 300               NMAE_train: 7.87%        NMAE_test: 6.0%          
epoch: 400               NMAE_train: 7.25%        NMAE_test: 5.52%         
epoch: 500               NMAE_train: 6.79%        NMAE_test: 5.19%         
epoch: 600               NMAE_train: 6.45%        NMAE_test: 4.95%         
epoch: 700               NMAE_train: 6.19%        NMAE_test: 4.77%         
epoch: 800               NMAE_train: 5.97%        NMAE_test: 4.62%         
epoch: 900               NMAE_train: 5.79%        NMAE_test: 4.51%         
epoch: 1000              NMAE_train: 5.64%        NMAE_test: 4.41%         
만남태양광: 8.79%                  
건국대학교 생명과학관 태양광: 20.02%                 

epoch: 600               NMAE_train: 5.44%        NMAE_test: 3.84%         
epoch: 700               NMAE_train: 5.14%        NMAE_test: 3.63%         
epoch: 800               NMAE_train: 4.89%        NMAE_test: 3.48%         
epoch: 900               NMAE_train: 4.69%        NMAE_test: 3.35%         
epoch: 1000              NMAE_train: 4.52%        NMAE_test: 3.25%         
월평정수장태양광: 0.0%                   
코레일대전태양광: 11.35%                 
도시환경이엔지태양광: 6.63%                  
종군교태양광: 6.38%                  
대전하수처리장태양광: 7.91%                  
Aggregation: 5.79%       
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
예측 지역 : 울산
epoch: 1                 NMAE_train: 16.83%       NMAE_test: 13.3%         
epoch: 100               NMAE_train: 11.67%       NMAE_test: 9.27%         
epoch: 200               NMAE_train: 9.75%        NMAE_test: 7.8%          
epoch: 300         

----------------------------------------
----------------------------------------
예측 지역 : 경북
epoch: 1                 NMAE_train: 15.74%       NMAE_test: 11.89%        
epoch: 100               NMAE_train: 11.37%       NMAE_test: 9.1%          
epoch: 200               NMAE_train: 9.72%        NMAE_test: 7.95%         
epoch: 300               NMAE_train: 8.81%        NMAE_test: 7.26%         
epoch: 400               NMAE_train: 8.22%        NMAE_test: 6.81%         
epoch: 500               NMAE_train: 7.8%         NMAE_test: 6.48%         
epoch: 600               NMAE_train: 7.49%        NMAE_test: 6.22%         
epoch: 700               NMAE_train: 7.24%        NMAE_test: 6.03%         
epoch: 800               NMAE_train: 7.03%        NMAE_test: 5.87%         
epoch: 900               NMAE_train: 6.85%        NMAE_test: 5.74%         
epoch: 1000              NMAE_train: 6.7%         NMAE_test: 5.63%         
은성쏠라21호태양광: 17.15%                 
유원성주태양광: 18.28%                 
은성

----------------------------------------
----------------------------------------
예측 지역 : 강원
epoch: 1                 NMAE_train: 15.06%       NMAE_test: 10.25%        
epoch: 100               NMAE_train: 11.24%       NMAE_test: 8.16%         
epoch: 200               NMAE_train: 9.8%         NMAE_test: 7.3%          
epoch: 300               NMAE_train: 9.01%        NMAE_test: 6.8%          
epoch: 400               NMAE_train: 8.5%         NMAE_test: 6.47%         
epoch: 500               NMAE_train: 8.12%        NMAE_test: 6.25%         
epoch: 600               NMAE_train: 7.84%        NMAE_test: 6.08%         
epoch: 700               NMAE_train: 7.6%         NMAE_test: 5.94%         
epoch: 800               NMAE_train: 7.4%         NMAE_test: 5.83%         
epoch: 900               NMAE_train: 7.22%        NMAE_test: 5.73%         
epoch: 1000              NMAE_train: 7.06%        NMAE_test: 5.64%         
공산쏠라 태양광: 12.87%                 
공산쏠라2 태양광: 12.79%                 
에너

...

----------------------------------------
----------------------------------------
예측 지역 : 서울
epoch: 1                 NMAE_train: 15.84%       NMAE_test: 15.12%        
epoch: 100               NMAE_train: 10.89%       NMAE_test: 10.22%        
epoch: 200               NMAE_train: 8.99%        NMAE_test: 8.44%         
epoch: 300               NMAE_train: 7.98%        NMAE_test: 7.52%         
epoch: 400               NMAE_train: 7.31%        NMAE_test: 6.93%         
epoch: 500               NMAE_train: 6.82%        NMAE_test: 6.5%          
epoch: 600               NMAE_train: 6.45%        NMAE_test: 6.19%         
epoch: 700               NMAE_train: 6.16%        NMAE_test: 5.94%         
epoch: 800               NMAE_train: 5.92%        NMAE_test: 5.75%         
epoch: 900               NMAE_train: 5.72%        NMAE_test: 5.59%         
epoch: 1000              NMAE_train: 5.56%        NMAE_test: 5.46%         
만남태양광: 6.79%                  
건국대학교 생명과학관 태양광: 8.44%                  

epoch: 600               NMAE_train: 5.06%        NMAE_test: 5.4%          
epoch: 700               NMAE_train: 4.8%         NMAE_test: 5.18%         
epoch: 800               NMAE_train: 4.59%        NMAE_test: 5.0%          
epoch: 900               NMAE_train: 4.41%        NMAE_test: 4.85%         
epoch: 1000              NMAE_train: 4.26%        NMAE_test: 4.74%         
월평정수장태양광: 7.9%                   
코레일대전태양광: 10.2%                  
도시환경이엔지태양광: 6.42%                  
종군교태양광: 9.24%                  
대전하수처리장태양광: 6.53%                  
Aggregation: 6.16%       
----------------------------------------
----------------------------------------
----------------------------------------
----------------------------------------
예측 지역 : 울산
epoch: 1                 NMAE_train: 15.8%        NMAE_test: 15.26%        
epoch: 100               NMAE_train: 10.53%       NMAE_test: 10.47%        
epoch: 200               NMAE_train: 8.58%        NMAE_test: 8.68%         
epoch: 300         

----------------------------------------
----------------------------------------
예측 지역 : 경북
epoch: 1                 NMAE_train: 15.38%       NMAE_test: 15.38%        
epoch: 100               NMAE_train: 11.15%       NMAE_test: 10.9%         
epoch: 200               NMAE_train: 9.52%        NMAE_test: 9.33%         
epoch: 300               NMAE_train: 8.6%         NMAE_test: 8.51%         
epoch: 400               NMAE_train: 8.01%        NMAE_test: 8.0%          
epoch: 500               NMAE_train: 7.59%        NMAE_test: 7.65%         
epoch: 600               NMAE_train: 7.27%        NMAE_test: 7.4%          
epoch: 700               NMAE_train: 7.01%        NMAE_test: 7.2%          
epoch: 800               NMAE_train: 6.79%        NMAE_test: 7.04%         
epoch: 900               NMAE_train: 6.61%        NMAE_test: 6.9%          
epoch: 1000              NMAE_train: 6.44%        NMAE_test: 6.79%         
은성쏠라21호태양광: 10.06%                 
유원성주태양광: 11.29%                 
은성

----------------------------------------
----------------------------------------
예측 지역 : 강원
epoch: 1                 NMAE_train: 14.85%       NMAE_test: 14.42%        
epoch: 100               NMAE_train: 11.11%       NMAE_test: 10.76%        
epoch: 200               NMAE_train: 9.69%        NMAE_test: 9.43%         
epoch: 300               NMAE_train: 8.9%         NMAE_test: 8.72%         
epoch: 400               NMAE_train: 8.37%        NMAE_test: 8.27%         
epoch: 500               NMAE_train: 7.99%        NMAE_test: 7.96%         
epoch: 600               NMAE_train: 7.69%        NMAE_test: 7.73%         
epoch: 700               NMAE_train: 7.44%        NMAE_test: 7.56%         
epoch: 800               NMAE_train: 7.23%        NMAE_test: 7.42%         
epoch: 900               NMAE_train: 7.05%        NMAE_test: 7.32%         
epoch: 1000              NMAE_train: 6.9%         NMAE_test: 7.23%         
공산쏠라 태양광: 21.86%                 
공산쏠라2 태양광: 12.6%                  
에너

----------------------------------------
----------------------------------------
예측 지역 : 전북
epoch: 1                 NMAE_train: 14.57%       NMAE_test: 15.96%        
epoch: 100               NMAE_train: 10.57%       NMAE_test: 11.42%        
epoch: 200               NMAE_train: 9.03%        NMAE_test: 9.78%         
epoch: 300               NMAE_train: 8.16%        NMAE_test: 8.91%         
epoch: 400               NMAE_train: 7.6%         NMAE_test: 8.36%         
epoch: 500               NMAE_train: 7.21%        NMAE_test: 7.99%         
epoch: 600               NMAE_train: 6.9%         NMAE_test: 7.71%         
epoch: 700               NMAE_train: 6.65%        NMAE_test: 7.49%         
epoch: 800               NMAE_train: 6.45%        NMAE_test: 7.32%         
epoch: 900               NMAE_train: 6.27%        NMAE_test: 7.17%         
epoch: 1000              NMAE_train: 6.12%        NMAE_test: 7.05%         
군산베스틸5기태양광: 10.82%                 
군산베스틸4기태양광: 11.81%                 

# bell shape 비율 통계

In [168]:
df_site_info_bell = pd.read_csv(address+'data_each_PV_info_20191201_20210814(bell).csv')

In [169]:
site_set = ["kwangjoo","sejong","choongnam","seoul","busan","daegoo","choongbook","daejeon","woolsan","incheon","kssouth","ksnorth","kyoungki","gangwon","jn","jb","jeju"]

In [177]:
for k in range(len(site_set)):

    site = site_set[k]

    if site == "gangwon":
        site_korean = "강원"
        weather_korean = "강릉"
    elif site =="seoul":
        site_korean = "서울"
        weather_korean = "서울"
    elif site =="choongnam":
        site_korean = "충남"
        weather_korean = "천안"
    elif site =="choongbook":
        site_korean = "충북"
        weather_korean = "청주"
    elif site =="sejong":
        site_korean = "세종"
        weather_korean = "세종"
    elif site =="busan":
        site_korean = "부산"
        weather_korean = "부산"
    elif site =="kyoungki":
        site_korean = "경기"
        weather_korean = "수원"
    elif site =="kwangjoo":
        site_korean = "광주"
        weather_korean = "광주"
    elif site =="daegoo":
        site_korean = "대구"
        weather_korean = "대구"
    elif site =="daejeon":
        site_korean = "대전"
        weather_korean = "대전"
    elif site =="woolsan":
        site_korean = "울산"
        weather_korean = "울산"
    elif site =="incheon":
        site_korean = "인천"
        weather_korean = "인천"
    elif site =="kssouth":
        site_korean = "경남"
        weather_korean = "김해시"
    elif site =="ksnorth":
        site_korean = "경북"
        weather_korean = "경주시"
    elif site =="jn":
        site_korean = "전남"
        weather_korean = "장흥"
    elif site =="jb":
        site_korean = "전북"
        weather_korean = "장수"
    elif site =="jeju":
        site_korean = "제주"
        weather_korean = "제주"
    print(site_korean)
    print(df_site_info_bell[df_site_info_bell['광역지역']==site_korean]['bell'].value_counts())
    print("=="*30)

광주
True     31
False     9
Name: bell, dtype: int64
세종
True     4
False    2
Name: bell, dtype: int64
충남
False    149
True     142
Name: bell, dtype: int64
서울
True     8
False    1
Name: bell, dtype: int64
부산
True     17
False     7
Name: bell, dtype: int64
대구
True     23
False     6
Name: bell, dtype: int64
충북
False    68
True     53
Name: bell, dtype: int64
대전
True    5
Name: bell, dtype: int64
울산
True     7
False    1
Name: bell, dtype: int64
인천
True     11
False     4
Name: bell, dtype: int64
경남
True     115
False     57
Name: bell, dtype: int64
경북
True     154
False    123
Name: bell, dtype: int64
경기
True     88
False    32
Name: bell, dtype: int64
강원
True     101
False     57
Name: bell, dtype: int64
전남
True     300
False    219
Name: bell, dtype: int64
전북
False    206
True     111
Name: bell, dtype: int64
제주
True     377
False     29
Name: bell, dtype: int64


In [159]:
df_site_info_bell['광역지역'].value_counts()

전남    519
제주    406
전북    317
충남    291
경북    277
경남    172
강원    158
충북    121
경기    120
광주     40
대구     29
부산     24
인천     15
서울      9
울산      8
세종      6
대전      5
Name: 광역지역, dtype: int64

# 들로네로 체크

In [114]:
data_PV_Info_del = pd.read_csv(address+'PV_delaunay_tri(bell).csv',index_col=0)

In [128]:
df_site_num_bell = df_site_info[df_site_info['bell']]['번호'].values

In [ ]:
def check_isin(input_tri,cluster):
    tf = False
    for i in range(len(cluster)):
        if np.isin(cluster[i],input_tri).all():
            tf = True
            
    return tf

In [ ]:
def check_delaunay(tf_array,n):
    C=np.equal(A,tf_array[n])
    D=np.zeros((C.shape[0]),dtype=bool)
    for i,TF in enumerate(C):
        if C[i][0] and C[i][1] and C[i][2]:
            D[i]= True
        else : D[i]= False
            
    return D

In [ ]:
# site_set = ["seoul","choongnam","choongbook","busan","daegoo","daejeon","woolsan","incheon","kssouth","ksnorth","kyoungki","gangwon"]
site_set = ["kwangjoo","sejong","choongnam","seoul","busan","daegoo","choongbook","daejeon","woolsan","incheon","kssouth","ksnorth","kyoungki","gangwon","jn","jb","jeju"]

test_month_array =['19_12','20_1','20_2','20_3','20_4','20_5','20_6','20_7','20_8',
                   '20_9','20_10','20_11','20_12','21_1','21_2','21_3','21_4','21_5',
                   '21_6','21_7','21_8']
epsilon = 10e-5

for p in range(len(test_month_array)):
    site_forecast_agg_result_array = []
    test_month = test_month_array[p]
    print("-"*80)
    print("-"*80)
    print("-"*80)
    print("-"*80)
    print("Test Month: ",test_month)
    print("-"*80)
    print("-"*80)
    print("-"*80)
    print("-"*80)
    for k in range(len(site_set)):

        site = site_set[k]

        if site == "gangwon":
            site_korean = "강원"
            weather_korean = "강릉"
        elif site =="seoul":
            site_korean = "서울"
            weather_korean = "서울"
        elif site =="choongnam":
            site_korean = "충남"
            weather_korean = "천안"
        elif site =="choongbook":
            site_korean = "충북"
            weather_korean = "청주"
        elif site =="sejong":
            site_korean = "세종"
            weather_korean = "세종"
        elif site =="busan":
            site_korean = "부산"
            weather_korean = "부산"
        elif site =="kyoungki":
            site_korean = "경기"
            weather_korean = "수원"
        elif site =="kwangjoo":
            site_korean = "광주"
            weather_korean = "광주"
        elif site =="daegoo":
            site_korean = "대구"
            weather_korean = "대구"
        elif site =="daejeon":
            site_korean = "대전"
            weather_korean = "대전"
        elif site =="woolsan":
            site_korean = "울산"
            weather_korean = "울산"
        elif site =="incheon":
            site_korean = "인천"
            weather_korean = "인천"
        elif site =="kssouth":
            site_korean = "경남"
            weather_korean = "김해시"
        elif site =="ksnorth":
            site_korean = "경북"
            weather_korean = "경주시"
        elif site =="jn":
            site_korean = "전남"
            weather_korean = "장흥"
        elif site =="jb":
            site_korean = "전북"
            weather_korean = "장수"
        elif site =="jeju":
            site_korean = "제주"
            weather_korean = "제주"
            
        data_PV_Info_delaunay = pd.read_csv(address+'PV_delaunay_tri(bell).csv')
        PV_site_isin = data_PV_Info_delaunay["광역지역"]==site_korean
        data_PV_Info_City = data_PV_Info_delaunay[PV_site_isin]
        data_PV_Info_City_Array = np.empty((1,len(data_PV_Info_City.values[0])),dtype=object)

        A = data_PV_Info_City[["지점0","지점1","지점2"]].values
        for i,li in enumerate(A):
            A[i] = sorted(li)
        cluster_delaunay = np.empty((1,3),dtype=object)
        for i,li in enumerate(A):
            if not check_isin(li,cluster_delaunay):
                cluster_delaunay=np.append(cluster_delaunay,li.reshape(1,3),axis=0)
        cluster_delaunay=cluster_delaunay[1:]

        
        
        
        cluster_delaunay_mat = np.empty((np.shape(cluster_delaunay)[0],np.shape(cluster_delaunay)[1]+1),dtype=object)
        data_PV_Info_City_Array = np.empty((1,len(data_PV_Info_City.values[0])),dtype=object)
        for i,cluster in enumerate(cluster_delaunay):
            delaunay_set=check_delaunay(cluster_delaunay,i)
            cluster_delaunay_mat[i][:3] = cluster_delaunay[i]
            cluster_delaunay_mat[i][3] = len(data_PV_Info_City.values[delaunay_set])
            data_PV_Info_City_Array = np.append(data_PV_Info_City_Array,data_PV_Info_City.values[delaunay_set],axis=0)

        data_PV_Info_City_Array = data_PV_Info_City_Array[1:]

        
         
        whole = 0
        for m,ele in enumerate(cluster_delaunay_mat):
        

            info_num_array = data_PV_Info_City_Array[:,1][whole:whole+ele[-1]]
            info_add_array = data_PV_Info_City_Array[:,2][whole:whole+ele[-1]]
            info_cap_array = data_PV_Info_City_Array[:,3][whole:whole+ele[-1]]
            info_lat_array = data_PV_Info_City_Array[:,5][whole:whole+ele[-1]]
            info_column_name = ['Number','Address','Latitude and Longitude','Capacity']

            info_set = np.vstack((info_num_array,info_add_array,info_lat_array,info_cap_array)).T

            df_site_info =pd.DataFrame(info_set, columns = info_column_name)
#             df_site_info.to_csv('PV_'+site+'_info.csv',encoding='utf-8-sig')

            column_name=['date','hour']
            print(site_korean)
            print(len(data_PV_Info_City))


            data_PV_Info = pd.read_csv(address+'new_PV_'+site+'_info.csv',index_col=0)
            data_PV_Value = pd.read_csv(address+'new_PV_'+site+'_value.csv',index_col=0)


            isin_cluster = data_PV_Info["Number"].isin(data_PV_Info_City_Array[whole:whole+ele[-1]][:,1].astype(int))
            print(whole,whole+ele[-1])
            data_PV_Info =data_PV_Info[isin_cluster]
            data_PV_Value = data_PV_Value[np.append(["date","hour"],data_PV_Info_City_Array[whole:whole+ele[-1]][:,1].astype(str))]

            """여기서부터 추가된 코드"""
            month = [31, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31,28,31,30,31,30,31,14] #여기서는 2019.12.01~2020.11.24
            month_test = p # 12월이 test인 경우
            train_len1 = sum(month[:month_test])
            train_len2 = sum(month[month_test+1:])
            train_len = train_len1 + train_len2

            """여기까지 추가된 코드"""

            """여기서부터 주석처리된 코드"""
        #     train_len    = 335 #여기서는 2020.01.01~2020.11.30까지가 Training (전체 335일)
            """여기까지 주석처리된 코드"""

            # 추가데이터에서 2019.12.01~2020.10.31까지 Training으로 쓴다면 336일
            train_input  = []
            train_output = []
            test_input   = []
            test_output  = []

            # Input Data (입력값을 배열로 만드는 과정, 추가데이터에서 2019.12.01~2020.11.24는 총 360일이니까 아래 366을 360으로 변경)
            data_Weather_1 = pd.read_csv(address_weather+'OBS_ASOS_TIM_merged_'+cluster_delaunay_mat[m][0]+'_imputed.csv')
            data_Temp_1 = np.nan_to_num(np.reshape(np.array(data_Weather_1['Temp']), (623,24)))/40
            data_Humid_1 = np.nan_to_num(np.reshape(np.array(data_Weather_1['Humid']), (623,24)))/100
            data_Wind_1 = np.nan_to_num(np.reshape(np.array(data_Weather_1['Wind']), (623,24)))/20
            data_Cloud_1 = np.nan_to_num(np.reshape(np.array(data_Weather_1['Cloud']), (623,24)))/10



            data_Weather_2 = pd.read_csv(address_weather+'OBS_ASOS_TIM_merged_'+cluster_delaunay_mat[m][1]+'_imputed.csv')
            data_Temp_2 = np.nan_to_num(np.reshape(np.array(data_Weather_2['Temp']), (623,24)))/40
            data_Humid_2 = np.nan_to_num(np.reshape(np.array(data_Weather_2['Humid']), (623,24)))/100
            data_Wind_2 = np.nan_to_num(np.reshape(np.array(data_Weather_2['Wind']), (623,24)))/20
            data_Cloud_2 = np.nan_to_num(np.reshape(np.array(data_Weather_2['Cloud']), (623,24)))/10


            data_Weather_3 = pd.read_csv(address_weather+'OBS_ASOS_TIM_merged_'+cluster_delaunay_mat[m][2]+'_imputed.csv')
            data_Temp_3 = np.nan_to_num(np.reshape(np.array(data_Weather_3['Temp']), (623,24)))/40
            data_Humid_3 = np.nan_to_num(np.reshape(np.array(data_Weather_3['Humid']), (623,24)))/100
            data_Wind_3 = np.nan_to_num(np.reshape(np.array(data_Weather_3['Wind']), (623,24)))/20
            data_Cloud_3 = np.nan_to_num(np.reshape(np.array(data_Weather_3['Cloud']), (623,24)))/10


            Input = np.concatenate((data_Temp_1,data_Temp_2,data_Temp_3, data_Humid_1,data_Humid_2,data_Humid_3,
                                    data_Wind_1,data_Wind_2,data_Wind_3, data_Cloud_1,data_Cloud_2,data_Cloud_3), axis=1)

            # Output Data (출력 태양광 데이터를 배열로 만드는 과정)

            for l in range(len(data_PV_Info)):
                Num = data_PV_Info['Number'].values[l]
                Cap = data_PV_Info['Capacity'].values[l]

                data_PV24 = np.zeros((623, 24)) # 여기도 360으로 변경
                j = 0
                for i in range(len(data_PV_Value)):
                    if i != 0 and data_PV_Value['date'][i] != data_PV_Value['date'][i-1]:
                        j += 1
                    h = int(data_PV_Value['hour'][i] - 1)
                    data_PV24[j][h] = data_PV_Value[str(Num)][i]

                if l==0: Output = data_PV24/Cap
                else: Output = np.append(Output, data_PV24/Cap, axis=1)

            """변경된 코드"""        
            if p == len(test_month_array)-1:
                train_input = Input[:train_len1]
                train_output = Output[:train_len1]
            else:
                train_input = np.append(Input[:train_len1], Input[-train_len2:], axis=0)
                train_output = np.append(Output[:train_len1], Output[-train_len2:], axis=0)
            test_input = Input[train_len1:train_len1 + month[month_test]]
            test_output = Output[train_len1:train_len1 + month[month_test]]          


        """변경된 코드"""


        # Model Construction (뉴런 개수와 레이어 수는 바꿔보면서 해도 됩니다)

        in_size        = len(train_input[0])
        hidden_size    = 128
        out_size       = len(train_output[0])
        batch_size     = 32
        learning_rate  = 1e-4

        class MLP(nn.Module):
            def __init__(self):
                super(MLP, self).__init__()
                self.fc_in  = nn.Linear(in_size, hidden_size)
                self.fc_h1  = nn.Linear(hidden_size, hidden_size)
                self.fc_h2  = nn.Linear(hidden_size, hidden_size)
                self.fc_out = nn.Linear(hidden_size, out_size)

                torch.nn.init.xavier_uniform_(self.fc_in.weight)
                torch.nn.init.xavier_uniform_(self.fc_h1.weight)
                torch.nn.init.xavier_uniform_(self.fc_h2.weight)
                torch.nn.init.xavier_uniform_(self.fc_out.weight)

            def forward(self, x):
                x = F.relu(self.fc_in(x))
                x = F.relu(self.fc_h1(x))
                x = F.relu(self.fc_h2(x))
                out = F.relu(self.fc_out(x))
                return out

        def train_net(model, batch, optimizer):
            x, y = batch[0], batch[1]
            loss = F.mse_loss(model.forward(x), y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Model Training (Total epoch 수도 적절하게 변경)
        # NMAE_test를 보면 오차가 2~3%대까지 내려갈 수 있는데 이는 밤 시간 데이터가 포함된 오차
        # 우리 관심사인 낮 시간 (이용률 = 발전량/설비용량 이 10% 이상인 것)에서의 오차는 이보다 클 수 있으니 주의

        total_epoch    = 1000
        print_interval = 100

        model = MLP()

        total_batch = int(train_len/batch_size) + 1
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        train_predict = []
        test_predict = []
        NMAE_train = []
        NMAE_test  = []
        print("-"*40)
        print("-"*40)
        print("예측 지역 :",site_korean)
        for epoch in range(total_epoch):
            for i in range(total_batch):
                batch_x = torch.tensor(train_input[batch_size*i:batch_size*(i+1),:], dtype=torch.float)
                batch_y = torch.tensor(train_output[batch_size*i:batch_size*(i+1),:], dtype=torch.float)
                batch = [batch_x, batch_y]
                train_net(model, batch, optimizer)

            if epoch == 0 or (epoch+1) % print_interval == 0:
                train_predict += [model.forward(torch.tensor(train_input, dtype=torch.float)).detach().numpy()]
                test_predict  += [model.forward(torch.tensor(test_input, dtype=torch.float)).detach().numpy()]
                NMAE_train += [round(100*np.mean(np.abs(train_output - train_predict)),2)]
                NMAE_test  += [round(100*np.mean(np.abs(test_output - test_predict)),2)]

                print("epoch: {}".format(epoch+1).ljust(25), end="")
                print("NMAE_train: {}%".format(NMAE_train[-1]).ljust(25), end="")
                print("NMAE_test: {}%".format(NMAE_test[-1]).ljust(25))

        i = np.argmin(NMAE_test) # test를 validation처럼 활용한 것

        # 아래는 데이터를 각 지역별로 쪼개서 오차 확인하고, 쪼갠 데이터를 Aggregation 한 후 오차 확인하는 것
        # 낮 시간에서의 오차를 5%대로 만드는 것이 목표
        test_output_each = []
        test_predict_each = []
        test_output_agg = np.zeros((np.size(test_output,0), 24))
        test_predict_agg = np.zeros((np.size(test_predict[i],0), 24))
        for j in range(len(data_PV_Info)):
            test_output_each += [test_output[:,24*j:24*(j+1)]]
            test_predict_each += [test_predict[i][:,24*j:24*(j+1)]]
            test_output_agg += data_PV_Info['Capacity'][j]*test_output_each[j]
            test_predict_agg += data_PV_Info['Capacity'][j]*test_predict_each[j]
        test_output_agg = test_output_agg/sum(data_PV_Info['Capacity'])
        test_predict_agg = test_predict_agg/sum(data_PV_Info['Capacity'])

        site_forecast_result_array = []
        for j in range(len(data_PV_Info)):
            test_object = test_output_each[j] > 0.1 # 이용률이 10% 이상인 것만 선정
            NMAE_test_object = round(100*np.sum(test_object*np.abs(test_output_each[j] - test_predict_each[j]))/(np.sum(test_object)+epsilon),2)
            print(data_PV_Info['Address'][j] + ": {}%".format(NMAE_test_object).ljust(25))
            site_forecast_result_array.append([data_PV_Info['Address'][j], "{}%".format(NMAE_test_object)])
        
        if not os.path.isdir(address+'광역예측결과_논문_들로네/{}월'.format(test_month)):
            os.mkdir(address+'광역예측결과_논문_들로네/{}월'.format(test_month))    
        
        df_site_forecast_result = pd.DataFrame(site_forecast_result_array, columns=['each Address','NMAE'])
        df_site_forecast_result.to_csv(address+'광역예측결과_논문_들로네/{}월/{}{}_NMAE_each_result.csv'.format(test_month,test_month,site_korean),encoding='utf-8-sig')

        test_object = test_output_agg > 0.1 # Aggregation 후에도 이용률이 10% 이상인 것만 선정
        NMAE_test_object = round(100*np.sum(test_object*np.abs(test_output_agg - test_predict_agg))/(np.sum(test_object)+epsilon),2)
        print("Aggregation: {}%".format(NMAE_test_object).ljust(25))

        site_forecast_agg_result_array.append([site_korean,"{}%".format(NMAE_test_object)])

        print("-"*40)
        print("-"*40)
        # 다른 궁금한 점은 언제든 질문~
    
#     if not os.path.isdir(address+'광역예측결과_Noise없음/{}/{}월/'.format(site_korean,test_month)):
#         os.mkdir(address+'광역예측결과_Noise없음/{}/{}월/'.format(site_korean,test_month))  

    df_site_agg_result = pd.DataFrame(site_forecast_agg_result_array, columns=['Wide Address','NMAE'])
    df_site_agg_result.to_csv(address+'광역예측결과_논문_들로네/{}월/{}_Wide_NMAE_result.csv'.format(test_month,test_month),encoding='utf-8-sig')
